In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score, confusion_matrix, mean_squared_error, auc, roc_curve, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.model_selection import cross_validate, RandomizedSearchCV


In [2]:
df = pd.read_json('./data.json', lines=True)
labels = pd.read_csv('./data.info', sep=',')

In [3]:
def get_dataframe(df):
    transcripts = df.columns
    dataframes = []
    for transcript in transcripts:
        df_curr_transcript = df.loc[:, transcript]
        df_curr_transcript.dropna(inplace=True)
        df_curr_transcript = list(df_curr_transcript)
        
        for position in df_curr_transcript:
            #position here is a dictionary
            for k1, v1, in position.items():
                order = list(v1.keys())[0]
                for k2, v2 in v1.items():
                    left_order = k2[:3]
                    centre_order = k2[1:3] + k2[5]
                    right_order = k2[2] + k2[5:]
                    
                    n = len(v2)

                    left_dwelling_t = np.zeros(n)
                    left_sd = np.zeros(n)
                    left_mean = np.zeros(n)

                    centre_dwelling_t = np.zeros(n)
                    centre_sd = np.zeros(n)
                    centre_mean = np.zeros(n)

                    right_dwelling_t = np.zeros(n)
                    right_sd = np.zeros(n)
                    right_mean = np.zeros(n)

                    for i in range(n):
                        left_dwelling_t[i] = v2[i][0]
                        left_sd[i] = v2[i][1]
                        left_mean[i] = v2[i][2]

                        centre_dwelling_t[i] = v2[i][3]
                        centre_sd[i] = v2[i][4]
                        centre_mean[i] = v2[i][5]
                        
                        right_dwelling_t[i] = v2[i][6]
                        right_sd[i] = v2[i][7]
                        right_mean[i] = v2[i][8]

                    left_dwelling_t_median = np.median(left_dwelling_t)
                    left_sd_median = np.median(left_sd)
                    left_mean_median = np.median(left_mean)

                    centre_dwelling_t_median = np.median(centre_dwelling_t)
                    centre_sd_median = np.median(centre_sd)
                    centre_mean_median = np.median(centre_mean)
                    
                    right_dwelling_t_median = np.median(right_dwelling_t)
                    right_sd_median = np.median(right_sd)
                    right_mean_median = np.median(right_mean)
                    
                    left_dwelling_t_log = np.mean(np.log(left_dwelling_t))
                    left_sd_log = np.mean(np.log(left_sd))
                    left_mean_log = np.mean(np.log(left_mean))

                    centre_dwelling_t_log = np.mean(np.log(centre_dwelling_t))
                    centre_sd_log = np.mean(np.log(centre_sd))
                    centre_mean_log =np.mean(np.log(centre_mean))
                    
                    right_dwelling_t_log = np.mean(np.log(right_dwelling_t))
                    right_sd_log = np.mean(np.log(right_sd))
                    right_mean_log = np.mean(np.log(right_mean))
 
                    #The interquartile range, often denoted “IQR”, is a way to measure the spread of the middle 50% of a dataset. It is calculated as the difference between the first quartile* (the 25th percentile) and the third quartile (the 75th percentile) of a dataset. 

                    # left_dwelling_t_q3,left_dwelling_t_q1 = np.percentile(left_dwelling_t,[75 ,25])
                    # left_dwelling_t_iqr =  left_dwelling_t_q3 -  left_dwelling_t_q1

                    # left_sd_q3, left_sd_q1= np.percentile(left_sd,[75 ,25])
                    # left_sd_iqr = left_sd_q3 -left_sd_q1

                    # left_mean_q3, left_mean_q1 = np.percentile(left_mean,[75 ,25])
                    # left_mean_iqr = left_mean_q3 -left_mean_q1

                    # centre_dwelling_t_q3, centre_dwelling_t_q1= np.percentile(centre_dwelling_t,[75 ,25])
                    # centre_dwelling_t_iqr = centre_dwelling_t_q3- centre_dwelling_t_q1

                    # centre_sd_q3,centre_sd_q1 = np.percentile(centre_sd,[75 ,25])
                    # centre_sd_iqr = centre_sd_q3 - centre_sd_q1

                    # centre_mean_q3, centre_mean_q1 = np.percentile(centre_mean,[75 ,25])
                    # centre_mean_iqr = centre_mean_q3 - centre_mean_q1
                    
                    # right_dwelling_t_q3, right_dwelling_t_q1 = np.percentile(right_dwelling_t,[75 ,25])
                    # right_dwelling_t_iqr = right_dwelling_t_q3 - right_dwelling_t_q1

                    # right_sd_q3, right_sd_q1= np.percentile(right_sd,[75 ,25])
                    # right_sd_iqr = right_sd_q3 - right_sd_q1

                    # right_mean_q3, right_mean_q1 = np.percentile(right_mean,[75 ,25])
                    # right_mean_iqr = right_mean_q3 - right_mean_q1

                    #  'left_sd_t_iqr' : [left_sd_iqr], 'left_mean_t_iqr' : [left_mean_iqr], 'centre_dwelling_iqr_t' : [centre_dwelling_t_iqr],
                    #                         'centre_sd_iqr' : [centre_sd_iqr], 'centre_mean_iqr' : [centre_mean_iqr], 'right_dwelling_iqr': [right_dwelling_t_iqr],
                    #                         'right_sd_iqr' : [right_sd_iqr], 'right_mean_iqr' : [right_mean_iqr]


                curr_dataframe = pd.DataFrame({'transcript_id':[transcript], 'order': [order], 'curr_pos': [k1], 'left_order': [left_order], 'median_left_dwelling_t': [left_dwelling_t_median], 'median_left_sd': [left_sd_median], 
                'median_left_mean': [left_mean_median], 'centre_order': [centre_order], 'median_centre_dwelling_t': [centre_dwelling_t_median], 'median_centre_sd': [centre_sd_median], 'median_centre_mean': [centre_mean_median], 
                                            'right_order': [right_order], 'median_right_dwelling_t': [right_dwelling_t_median], 'median_right_sd': [right_sd_median], 'median_right_mean': [right_mean_median],'left_dwelling_t_log' :[left_dwelling_t_log],
                                            'left_sd_log':[left_sd_log],'left_mean_log':[left_mean_log],'centre_dwelling_t_log':[centre_dwelling_t_log],'centre_sd_log':[centre_sd_log],'centre_mean_log':[centre_mean_log],'right_dwelling_t_log':[right_dwelling_t_log],
                                            'right_sd_log':[right_sd_log],'right_mean_log':[right_mean_log]

                                           
                                            })   
                dataframes.append(curr_dataframe)
    final_df = pd.concat(dataframes)
    final_df['curr_pos'] = final_df['curr_pos'].astype('int64')
    return final_df 

In [4]:
df1 = get_dataframe(df)

In [5]:
df1 = pd.merge(df1, labels, how = 'left', left_on = ['transcript_id', 'curr_pos'], right_on = ['transcript_id', 'transcript_position'])
df1 = df1.drop(['transcript_position'], axis='columns')

In [6]:
df2 = df1.copy()
df2['left_1'] = df2['left_order'].apply(lambda x: x[0])
df2['left_2'] = df2['left_order'].apply(lambda x: x[1])
df2['left_3'] = df2['left_order'].apply(lambda x: x[2])


df2['centre_1'] = df2['centre_order'].apply(lambda x: x[0])
df2['centre_2'] = df2['centre_order'].apply(lambda x: x[1])
df2['centre_3'] = df2['centre_order'].apply(lambda x: x[2])


df2['right_1'] = df2['right_order'].apply(lambda x: x[0])
df2['right_2'] = df2['right_order'].apply(lambda x: x[1])
df2['right_3'] = df2['right_order'].apply(lambda x: x[2])

In [7]:
#categorical data
categorical_cols = ['left_1', 'left_2', 'left_3',
       'centre_1', 'centre_2', 'centre_3', 'right_1',
       'right_2', 'right_3']

#import pandas as pd
df3 = pd.get_dummies(df2, columns = categorical_cols)

In [8]:
expected_columns = ['left_1_A', 'left_1_C', 'left_1_G', 'left_1_T',
                    'left_2_A', 'left_2_C', 'left_2_G', 'left_2_T', 
                    'left_3_A', 'left_3_C', 'left_3_G', 'left_3_T', 
                    'centre_1_A', 'centre_1_C', 'centre_1_G', 'centre_1_T', 
                    'centre_2_A', 'centre_2_C', 'centre_2_G', 'centre_2_T', 
                    'centre_3_A', 'centre_3_C', 'centre_3_G', 'centre_3_T', 
                    'right_1_A', 'right_1_C', 'right_1_G', 'right_1_T', 
                    'right_2_A', 'right_2_C', 'right_2_G', 'right_2_T', 
                    'right_3_A', 'right_3_C', 'right_3_G', 'right_3_T']

In [9]:
df3['a_count_l'] = df3['left_order'].str.count('A')
df3['c_count_l'] = df3['left_order'].str.count('C')
df3['g_count_l'] = df3['left_order'].str.count('G')
df3['t_count_l'] = df3['left_order'].str.count('T')

df3['a_count_c'] = df3['centre_order'].str.count('A')
df3['c_count_c'] = df3['centre_order'].str.count('C')
df3['g_count_c'] = df3['centre_order'].str.count('G')
df3['t_count_c'] = df3['centre_order'].str.count('T')

df3['a_count_r'] = df3['right_order'].str.count('A')
df3['c_count_r'] = df3['right_order'].str.count('C')
df3['g_count_r'] = df3['right_order'].str.count('G')
df3['t_count_r'] = df3['right_order'].str.count('T')

In [10]:
df3

,transcript_id,order,curr_pos,left_order,median_left_dwelling_t,median_left_sd,median_left_mean,centre_order,median_centre_dwelling_t,median_centre_sd,...,g_count_l,t_count_l,a_count_c,c_count_c,g_count_c,t_count_c,a_count_r,c_count_r,g_count_r,t_count_r
0,ENST00000000233,AAGACCA,244,AAG,0.00697,3.73,125.0,AGC,0.007970,6.650,...,1,0,1,1,1,0,1,1,1,0
1,ENST00000000233,CAAACTG,261,CAA,0.00564,2.88,110.0,AAT,0.005885,3.000,...,0,0,2,0,0,1,1,0,1,1
2,ENST00000000233,GAAACAG,316,GAA,0.00631,2.65,106.0,AAA,0.006310,3.780,...,1,0,3,0,0,0,2,0,1,0
3,ENST00000000233,AGAACAT,332,AGA,0.00902,5.73,130.0,GAA,0.007320,2.635,...,1,0,2,0,1,0,2,0,0,1
4,ENST00000000233,AGGACAA,368,AGG,0.00896,6.52,118.0,GGA,0.010500,5.660,...,2,0,1,0,2,0,2,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,GGGACAT,1348,GGG,0.00817,3.20,118.0,GGA,0.005310,4.580,...,3,0,1,0,2,0,1,0,1,1
121834,ENST00000641834,CTGACAC,1429,CTG,0.00618,3.69,112.0,TGA,0.009600,9.140,...,1,1,1,0,1,1,1,1,1,0
121835,ENST00000641834,TGGACAC,1531,TGG,0.00697,3.83,114.0,GGA,0.005725,4.440,...,2,1,1,0,2,0,1,1,1,0
121836,ENST00000641834,CTGACCA,1537,CTG,0.00660,3.16,110.0,TGC,0.006810,5.790,...,1,1,0,1,1,1,1,1,1,0


In [11]:
added_cols = []
df3_cols = df3.columns
for col in expected_columns:
    if col not in df3_cols:
        df3[col] = [0 for i in range(len(df3))]
        added_cols.append(col)

In [12]:
ordered_columns = ['transcript_id', 'order', 'curr_pos', 'left_order',
       'median_left_dwelling_t', 'median_left_sd', 'median_left_mean',
       'centre_order', 'median_centre_dwelling_t', 'median_centre_sd',
       'median_centre_mean', 'right_order', 'median_right_dwelling_t',
       'median_right_sd', 'median_right_mean', 'gene_id', 
       'left_1_A', 'left_1_C', 'left_1_G', 'left_1_T',
                    'left_2_A', 'left_2_C', 'left_2_G', 'left_2_T', 
                    'left_3_A', 'left_3_C', 'left_3_G', 'left_3_T', 
                    'centre_1_A', 'centre_1_C', 'centre_1_G', 'centre_1_T', 
                    'centre_2_A', 'centre_2_C', 'centre_2_G', 'centre_2_T', 
                    'centre_3_A', 'centre_3_C', 'centre_3_G', 'centre_3_T', 
                    'right_1_A', 'right_1_C', 'right_1_G', 'right_1_T', 
                    'right_2_A', 'right_2_C', 'right_2_G', 'right_2_T', 
                    'right_3_A', 'right_3_C', 'right_3_G', 'right_3_T', 'label']
df3 = df3[ordered_columns]
    

In [26]:
df3.corr()

/var/folders/mj/jmtkwrpx2nz0ly9w5_prydc80000gn/T/ipykernel_33333/3136175663.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df3.corr()


,curr_pos,median_left_dwelling_t,median_left_sd,median_left_mean,median_centre_dwelling_t,median_centre_sd,median_centre_mean,median_right_dwelling_t,median_right_sd,median_right_mean,...,right_1_T,right_2_A,right_2_C,right_2_G,right_2_T,right_3_A,right_3_C,right_3_G,right_3_T,label
curr_pos,1.000000,-0.030458,-0.056420,-0.079536,-0.061505,-0.066936,-0.072692,-0.013180,-0.048267,0.042236,...,NaN,0.005227,-0.042481,NaN,0.036190,-0.002942,-0.003761,-0.023568,0.028570,-0.003338
median_left_dwelling_t,-0.030458,1.000000,0.086581,0.223414,0.062862,-0.001917,-0.078553,-0.013994,-0.089585,-0.029382,...,NaN,0.043790,0.071154,NaN,-0.114916,0.014833,0.031891,-0.028386,-0.016301,0.038591
median_left_sd,-0.056420,0.086581,1.000000,0.548429,0.131479,-0.012897,-0.062856,0.036375,0.006240,0.012369,...,NaN,0.012030,0.034689,NaN,-0.046393,-0.017468,0.028264,0.012852,-0.020741,0.037579
median_left_mean,-0.079536,0.223414,0.548429,1.000000,0.268725,0.032108,0.251254,0.064047,0.225425,0.006592,...,NaN,-0.060103,0.016040,NaN,0.046406,-0.026876,0.022018,0.015586,-0.008564,0.098427
median_centre_dwelling_t,-0.061505,0.062862,0.131479,0.268725,1.000000,0.148406,0.472182,0.149226,0.374171,-0.158283,...,NaN,-0.042437,0.033416,NaN,0.011137,-0.020753,0.055966,-0.012382,-0.018668,0.011494
median_centre_sd,-0.066936,-0.001917,-0.012897,0.032108,0.148406,1.000000,0.646091,0.097543,0.393781,-0.618829,...,NaN,0.025240,0.161892,NaN,-0.184585,0.008576,0.012698,0.022787,-0.041365,-0.020356
median_centre_mean,-0.072692,-0.078553,-0.062856,0.251254,0.472182,0.646091,1.000000,0.148195,0.654582,-0.336340,...,NaN,-0.154993,0.011235,NaN,0.149168,-0.059175,0.023328,0.005217,0.031410,0.065540
median_right_dwelling_t,-0.013180,-0.013994,0.036375,0.064047,0.149226,0.097543,0.148195,1.000000,0.091703,0.056734,...,NaN,0.004353,-0.172702,NaN,0.164587,0.001112,-0.072908,0.263716,-0.184999,0.034877
median_right_sd,-0.048267,-0.089585,0.006240,0.225425,0.374171,0.393781,0.654582,0.091703,1.000000,-0.364124,...,NaN,0.055628,0.036874,NaN,-0.093587,-0.108516,0.067753,0.049207,-0.002489,-0.001529
median_right_mean,0.042236,-0.029382,0.012369,0.006592,-0.158283,-0.618829,-0.336340,0.056734,-0.364124,1.000000,...,NaN,-0.146318,-0.572117,NaN,0.711339,-0.114043,0.013442,0.088757,0.014453,0.025659


In [13]:
genes = list(labels['gene_id'].unique())
train_genes, test_genes = train_test_split(genes, train_size = 0.8, random_state=42)
train_labels = labels[labels['gene_id'].isin(train_genes)]
test_labels = labels[labels['gene_id'].isin(test_genes)]

In [14]:
X_train = pd.merge(train_labels, df3, how='inner',left_on=['gene_id', 'transcript_id', 'transcript_position', 'label'], right_on=['gene_id', 'transcript_id', 'curr_pos', 'label'])
X_test = pd.merge(test_labels, df3, how='inner',left_on=['gene_id','transcript_id', 'transcript_position', 'label'], right_on=['gene_id', 'transcript_id', 'curr_pos', 'label'])

In [15]:
y_train = np.asarray(X_train['label'])
y_test = np.asarray(X_test['label'])

In [16]:
X_test = X_test.drop(['gene_id', 'transcript_id', 'transcript_position','label', 'order', 'left_order', 'centre_order', 'right_order'], axis=1)

In [17]:
X_train = X_train.drop(['gene_id', 'transcript_id', 'transcript_position','label', 'order', 'left_order', 'centre_order', 'right_order'], axis=1)
y_train = y_train

In [18]:
oversample = SMOTE(random_state=42)
X_training_smote, y_training_smote = oversample.fit_resample(X_train, y_train)

In [19]:
scaler= MinMaxScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

rf_clf = RandomForestClassifier()
rf_clf.fit(scaled_X_train, y_train)
rf_y_pred = rf_clf.predict(scaled_X_test)
rf_y_score = rf_clf.predict_proba(scaled_X_test)[:, 1]


In [20]:
print(f"rf AUCROC Score: {roc_auc_score(y_test, rf_y_score)}")
print(f"rf PR AUC: {average_precision_score(y_test, rf_y_score)}")

rf AUCROC Score: 0.8687202637667708
rf PR AUC: 0.39885175391691247


In [21]:
X_train.shape

(96839, 46)

In [23]:
from keras.models import Sequential
from keras.layers import Dense, Activation

from tensorflow.keras.metrics import AUC
pr_metric = AUC(curve='PR', num_thresholds=1000)

model = Sequential()
model.add(Dense(12, input_shape=(46,), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[pr_metric])
model.fit(scaled_X_train, y_train, epochs=10, batch_size=10)
y_pred = model.predict(scaled_X_test)

Epoch 1/10


2022-09-24 21:14:21.418079: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-24 21:14:21.645140: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9684/9684 [==============================] - 76s 8ms/step - loss: 0.1591 - auc_1: 0.1594
Epoch 2/10
9684/9684 [==============================] - 69s 7ms/step - loss: 0.1449 - auc_1: 0.2794
Epoch 3/10
9684/9684 [==============================] - 69s 7ms/step - loss: 0.1362 - auc_1: 0.3580
Epoch 4/10
9684/9684 [==============================] - 69s 7ms/step - loss: 0.1328 - auc_1: 0.3829
Epoch 5/10
9684/9684 [==============================] - 69s 7ms/step - loss: 0.1314 - auc_1: 0.3895
Epoch 6/10
9684/9684 [==============================] - 68s 7ms/step - loss: 0.1306 - auc_1: 0.3948
Epoch 7/10
9684/9684 [==============================] - 74s 8ms/step - loss: 0.1297 - auc_1: 0.4029
Epoch 8/10
9684/9684 [==============================] - 80s 8ms/step - loss: 0.1293 - auc_1: 0.4047
Epoch 9/10
9684/9684 [==============================] - 82s 8ms/step - loss: 0.1291 - auc_1: 0.4077
Epoch 10/10
 76/782 [=>............................] - ETA: 1s

2022-09-24 21:26:42.581313: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 2s 2ms/step


In [25]:
from sklearn.metrics import average_precision_score
average_precision_score(y_test, y_pred)

0.3827023175021523